# Curating a genome scale model

This notebook has been tested on [jprime.lbl.gov](jprime.lbl.gov) with the biodesign_3.7 kernel.

It starts with the model that gets output by the annotation_gr.ipynb notebook.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import IFrame
import numpy as np
import pandas as pd
import json
import urllib
import cobra
import cplex
import os
import requests
import collections
import itertools

import cobra.test
from cobra.flux_analysis import gapfill

# Getting and preparing the genome-scale model

## Load *R.opacus* NCBI model generated by CarveMe

In [3]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated_curated.xml")
model

Name,ropacus_curated_with_5_deleted_reactions
Memory address,0x07fdee01e5f90
Number of metabolites,1581
Number of reactions,2380
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


## Starting MEMOTE Output
Need to update

In [3]:
# IFrame('memotes/ropacus_curated_with_5_deleted_reactions.htm', 1500, 800)

# Look into Various methods to play with the GSM

#### follow simulating FBA documentation page 

In [6]:
model.objective = "Growth"
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 10.0
    model.medium = medium
    solution = model.optimize()
    print(solution)

<Solution 0.792 at 0x7f5423c80a50>


In [13]:
print(model.metabolites.get_by_id("pydam_c").name)
for r in model.metabolites.get_by_id("pydam_c").reactions:
    print(r.name, r)

Pyridoxamine
Pyridoxamine import PYDAMtpp: pydam_p --> pydam_c
Pyridoxal oxidase PYDXO: 2.0 h2o_c + nh4_c + 0.5 o2_c + pydx_c <-- 2.0 h2o2_c + pydam_c


### These are reactions in the reverse direction. Why are they there?

In [20]:
for r in [r for r in model.reactions if ('-->' not in r.reaction and '<=>' not in r.reaction)]:
    print(r)

ACACT5r: accoa_c + dcacoa_c <-- 3oddcoa_c + coa_c
BZSCD: bzsuccoa_c + fad_c <-- fadh2_c + phitcoa_c
MGt5: mg2_c <-- mg2_e
PPGKr: glc__D_c + pi_c <-- g6p_c + h2o_c
PYDXO: 2.0 h2o_c + nh4_c + 0.5 o2_c + pydx_c <-- 2.0 h2o2_c + pydam_c
SUCD1: fad_c + succ_c <-- fadh2_c + fum_c
SULR_1: 3.0 h2o_c + h2s_c + 3.0 nadp_c <-- 5.0 h_c + 3.0 nadph_c + so3_c


In [21]:
print(model.metabolites.get_by_id("fmn_c").name)
for r in model.metabolites.get_by_id("fmn_c").reactions:
    print(r.name, r)

FMN
NAD(P)H-flavin oxidoreductase FLDO: fmnh2_c + nadp_c --> fmn_c + h_c + nadph_c
FMN dependent monooxygenase  taurine  FDMO4_1: fmnRD_c + o2_c + taur_c --> amacald_c + fmn_c + h2o_c + h_c + so3_c
FMN reductase FMNRx: fmn_c + h_c + nadh_c --> fmnh2_c + nad_c
FMN reductase  NADPH dependent  FMNRy: fmn_c + h_c + nadph_c --> fmnRD_c + nadp_c
FMNH2-dependent monooxygenase FDMO: fmnh2_c + isetac_c + o2_c --> fmn_c + gcald_c + h2o_c + h_c + so3_c
FMN reductase FMNRx2: fmn_c + h_c + nadph_c --> fmnh2_c + nadp_c
FMN dependent monooxygenase  hexanesulfonate  FDMO3_1: fmnRD_c + hexs_c + o2_c --> fmn_c + h2o_c + h_c + hxal_c + so3_c
FmnRD dependent monooxygenase  methanesulfonate  FDMO_1: fmnRD_c + mso3_c + o2_c --> fald_c + fmn_c + h2o_c + h_c + so3_c
FMN dependent monooxygenase  butanesulfonate  FDMO2_1: buts_c + fmnRD_c + o2_c --> btal_c + fmn_c + h2o_c + h_c + so3_c
FMNH2-dependent monooxygenase (ethanesulfonate) FDMO3: ethso3_c + fmnh2_c + o2_c --> acald_c + fmn_c + h2o_c + h_c + so3_c
FMN 

What is the difference between a sink and an exchange?

Exchange reactions - are reactions that move metabolites across in silico compartments. These in silico compartments are representive of intra- and inter- cellular membranes.<br><br>
Sink reactions - The metabolites, produced in reactions that are outside of an ambit of the system or in unknown reactions, are supplied to the network with reversible sink reactions.<br><br>
Demand reactions - Irreversible reactions added to the model to consume metabolites that are deposited in the system.

In [ ]:
for r in model.boundary:
    print(r)